In [1]:
import numpy as np
import sys
print(sys.version)

3.7.0 (v3.7.0:1bf9cc5093, Jun 26 2018, 23:26:24) 
[Clang 6.0 (clang-600.0.57)]


In [2]:
import logging.config
from sklearn.utils import shuffle

import numpy as np
from os import path

In [3]:
import quandl

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras import Sequential, callbacks
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop

In [5]:
import torchsummary

In [6]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from DataLoader import DataLoader
from MarketData import QuandlMarketDataSource, RedditMarketDataSource, BloombergMarketDataSource

from learning.BasicLearning import RbfClassifier
from utilities import Constants
from utilities.Utilities import Utilities

In [7]:
key_dir = "/Users/tianyudu/Documents/UToronto/Course/ECO499/quandl_api.txt"
with open(key_dir, "r") as f:
    key = f.readline()

In [8]:
from Experiment import *

In [83]:
# if __name__ == '__main__':
#     item = 'JPM'
#     price_source = 'quandl'
#     processing_type = 'SVM'
#     sentiment_location = path.join(Constants.DATASETS_MARKET, 'Twitter/psenti.csv')
#     # technical analysis
#     processing(price_source, item, None, False, processing_type)
#     # technical analysis + Sentiment
#     sentiment_location = path.join(Constants.DATASETS_MARKET, 'FinArticles/psenti/all.results.csv')
#     processing(price_source, item, sentiment_location, False, processing_type)
#     sentiment_location = path.join(Constants.DATASETS_MARKET, 'FinArticles/psenti/reddit.results.csv')
#     processing(price_source, item, sentiment_location, False, processing_type)
#     # technical analysis + Sentiment + Mood
#     processing(price_source, item, sentiment_location, True, processing_type)

In [84]:
item = 'JPM'
price_source = 'quandl'
processing_type = 'SVM'
sentiment_location = path.join(Constants.DATASETS_MARKET, 'Twitter/psenti.csv')

In [87]:
x_data, y_data = get_data(False, None, price_source, item)

2019-10-31 17:47:28,374 - utilities - INFO - Loading Stock [JPM]...


/Users/tianyudu/Documents/UToronto/Course/ECO499/3_Replication_Market_Wisdom/src/PortfolioBasic/stockstats.py:387: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  lambda x: np.fabs(x - x.mean()).mean())


In [111]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=42)

In [112]:
y_train_2 = np.concatenate([(y_train==0).reshape(-1, 1), (y_train==1).reshape(-1, 1)], axis=1).astype(np.int)

In [113]:
y_train = Utilities.make_dual(y_train, 2)  # Convert to one-hot.
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [114]:
y_train_2 == y_train

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       ...,
       [ True,  True],
       [ True,  True],
       [ True,  True]])

In [68]:
market_model = build_model(x_train, model_type="Conv")
market_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 16, 200)           161600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 16, 32)            19232     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 32)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                

In [69]:
cbks = [callbacks.EarlyStopping(monitor='val_loss', patience=25)]
market_model.fit(x_train, y_train, batch_size=1000, callbacks=cbks, epochs=50, validation_split=0.25, shuffle=True, verbose=1)

Train on 562 samples, validate on 188 samples
Epoch 1/50
562/562 [==============================] - 7s 12ms/sample - loss: 0.6921 - accuracy: 0.5498 - val_loss: 0.6967 - val_accuracy: 0.5426
Epoch 2/50
562/562 [==============================] - 0s 706us/sample - loss: 0.6850 - accuracy: 0.5623 - val_loss: 0.6944 - val_accuracy: 0.4947
Epoch 3/50
562/562 [==============================] - 0s 837us/sample - loss: 0.6847 - accuracy: 0.5765 - val_loss: 0.7024 - val_accuracy: 0.4947
Epoch 4/50
562/562 [==============================] - 0s 663us/sample - loss: 0.6816 - accuracy: 0.5907 - val_loss: 0.7008 - val_accuracy: 0.4787
Epoch 5/50
562/562 [==============================] - 0s 563us/sample - loss: 0.6825 - accuracy: 0.5676 - val_loss: 0.7028 - val_accuracy: 0.4840
Epoch 6/50
562/562 [==============================] - 0s 752us/sample - loss: 0.6795 - accuracy: 0.5943 - val_loss: 0.7120 - val_accuracy: 0.4734
Epoch 7/50
562/562 [==============================] - 0s 651us/sample - loss: 0

In [70]:
y_result_prob = market_model.predict(x_test)

In [71]:
# y_result = Utilities.make_single_dimension(y_result_prob)
y_result = np.argmax(y_result_prob, axis=1)

In [72]:
Utilities.measure_performance(y_test, y_result)

2019-10-31 15:33:51,842 - utilities - INFO - 
              precision    recall  f1-score   support

           0      0.449     0.216     0.291       102
           1      0.602     0.818     0.693       148

    accuracy                          0.572       250
   macro avg      0.525     0.517     0.492       250
weighted avg      0.540     0.572     0.529       250

2019-10-31 15:33:51,849 - utilities - INFO - Macro F1 0.492
2019-10-31 15:33:51,853 - utilities - INFO - Micro F1 0.572


In [73]:
Utilities.measure_performance_auc(y_test, y_result, y_result)

2019-10-31 15:33:51,874 - utilities - INFO - Accurary: 0.572 and AUC 0.517


(0.572, 0.5166269210386858)

In [9]:
from MarketData import BloombergMarketDataSource
source = BloombergMarketDataSource()
stock="JPM"
price_source = "quandl"
sentiment_location = path.join(Constants.DATASETS_MARKET, "FinArticles/psenti/all.results.csv")

In [17]:
x_data, y_data = get_data(
    True,
    sentiment_location,
    price_source,
    stock)

2019-10-31 20:56:01,445 - utilities - INFO - Loading Stock [JPM]...


/Users/tianyudu/Documents/UToronto/Course/ECO499/3_Replication_Market_Wisdom/src/DataLoader.py:67: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  articles = articles.groupby(pd.TimeGrouper('D')).mean()
/Users/tianyudu/Documents/UToronto/Course/ECO499/3_Replication_Market_Wisdom/src/PortfolioBasic/stockstats.py:387: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  lambda x: np.fabs(x - x.mean()).mean())


In [18]:
foo = lambda x: np.reshape(x, (x.shape[0], x.shape[1], 1))

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=42)

In [22]:
pipeline = Pipeline([
        ['clf', RbfClassifier()]])
pipeline.fit(x_train, y_train)
y_result = pipeline.predict(x_test)
y_result_prob = pipeline.predict_proba(x_test)

2019-10-31 20:56:34,942 - utilities - INFO - Found 0 type with 338 records
2019-10-31 20:56:34,946 - utilities - INFO - Found 1 type with 412 records
2019-10-31 20:56:34,947 - learning - INFO - Searching classifier best parameters


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


2019-10-31 20:56:36,077 - learning - INFO - Best parameters found:
2019-10-31 20:56:36,078 - learning - INFO - 0.6586666666666666
2019-10-31 20:56:36,079 - learning - INFO - {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
2019-10-31 20:56:36,080 - learning - INFO - Creating calibrated...
2019-10-31 20:56:36,080 - learning - INFO - Training...


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.7